# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [44]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas
import requests


# Import API key
from api_keys import geoapify_key

In [45]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.47,82,92,20.20,TF,1690395956
1,1,grytviken,-54.2811,-36.5092,-1.58,95,95,3.90,GS,1690395956
2,2,ushuaia,-54.8000,-68.3000,1.81,64,40,5.66,AR,1690395957
3,3,arraial do cabo,-22.9661,-42.0278,21.92,88,75,11.32,BR,1690395957
4,4,karlovac,45.4872,15.5478,18.34,94,57,0.45,HR,1690395957


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [46]:
%%capture --no-display

 # Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',  # Longitude values from the DataFrame
    y='Lat',  # Latitude values from the DataFrame
    geo=True,  # Enable geographic projection
    tiles="EsriImagery",  # Map tile source (you can choose other tile sources as well)
    frame_width=800,
    frame_height=600,
    size="Humidity",  # Use 'Humidity' column from the DataFrame for point size
    scale=0.5,
)

# Display the map plot
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [47]:
# Criteria for your ideal weather condition (e.g., max temperature between 20°C and 30°C, humidity below 60%, and cloudiness below 50%)
ideal_conditions_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30) & 
    (city_data_df['Humidity'] < 60) & 
    (city_data_df['Cloudiness'] < 50)
].copy()

# Drop any rows with null values
ideal_conditions_df.dropna(inplace=True)

# Display sample data
ideal_conditions_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,portland,45.5234,-122.6762,24.44,59,20,4.63,US,1690395805
19,19,coromandel,-18.4733,-47.2003,27.19,20,0,1.00,BR,1690395904
21,21,ifrane,33.3333,-5.2500,24.15,41,0,4.19,MA,1690395960
34,34,colonia,50.9333,6.9500,21.65,48,0,5.66,DE,1690395719
76,76,vernon,50.2581,-119.2691,24.05,40,11,2.24,CA,1690395929


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
# Use the Pandas copy function to create a DataFrame called hotel_df
hotel_df = ideal_conditions_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,16,portland,45.5234,-122.6762,24.44,59,20,4.63,US,1690395805,
19,19,coromandel,-18.4733,-47.2003,27.19,20,0,1.00,BR,1690395904,
21,21,ifrane,33.3333,-5.2500,24.15,41,0,4.19,MA,1690395960,
34,34,colonia,50.9333,6.9500,21.65,48,0,5.66,DE,1690395719,
76,76,vernon,50.2581,-119.2691,24.05,40,11,2.24,CA,1690395929,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [49]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "type": "hotel",
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat= row["Lat"]
    lng= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distanceToPoint:{lng},{lat}<=8000"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params= params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
portland - nearest hotel: No hotel found
coromandel - nearest hotel: No hotel found
ifrane - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
vernon - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
susanville - nearest hotel: No hotel found
st. john's - nearest hotel: No hotel found
san juanito - nearest hotel: No hotel found
mount pearl - nearest hotel: No hotel found
yamba - nearest hotel: No hotel found
cranbrook - nearest hotel: No hotel found
cordoba - nearest hotel: No hotel found
tekeli - nearest hotel: No hotel found
oxapampa - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
ilabaya - nearest hotel: No hotel found
teguise - nearest hotel: No hotel found
urzhar - nearest hotel: No hotel found
kostyantynivka - nearest hotel: No hotel found
pavlovsk - nearest hotel: No hotel found
santa rosa de viterbo - nearest hotel: No hotel f

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,16,portland,45.5234,-122.6762,24.44,59,20,4.63,US,1690395805,No hotel found
19,19,coromandel,-18.4733,-47.2003,27.19,20,0,1.00,BR,1690395904,No hotel found
21,21,ifrane,33.3333,-5.2500,24.15,41,0,4.19,MA,1690395960,No hotel found
34,34,colonia,50.9333,6.9500,21.65,48,0,5.66,DE,1690395719,No hotel found
76,76,vernon,50.2581,-119.2691,24.05,40,11,2.24,CA,1690395929,No hotel found
86,86,happy valley-goose bay,53.3168,-60.3315,26.17,41,40,5.14,CA,1690395977,No hotel found
110,110,yellowknife,62.4560,-114.3525,24.29,35,40,8.49,CA,1690395988,No hotel found
131,131,susanville,40.4163,-120.6530,28.23,19,0,2.57,US,1690395975,No hotel found
161,161,st. john's,47.5649,-52.7093,29.66,42,40,6.69,CA,1690396003,No hotel found
170,170,san juanito,28.0000,-107.5833,27.11,18,49,4.29,MX,1690396005,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [50]:
%%capture --no-display

# Configure the map plot with hover information
map_plot = city_data_df.hvplot.points(
    x='Lng',  # Longitude values from the DataFrame
    y='Lat',  # Latitude values from the DataFrame
    geo=True,  # Enable geographic projection
    tiles="EsriImagery",  # Map tile source (you can choose other tile sources as well)
    frame_width=800,
    frame_height=600,
    size="Humidity",  # Use 'Humidity' column from the DataFrame for point size
    scale=0.5,
    hover_cols=["City", "Country", "Hotel Name", "Humidity"],  # Additional columns to display in hover tooltip
)

# Display the map plot
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)